In [1]:
import pandas as pd
import numpy as np
import os
import tensorflow as tf
from keras.applications.vgg16 import VGG16
from keras.models import Sequential
from keras.models import Model
from keras.layers import Input, Lambda, Dense, Flatten
import cv2
from tensorflow import keras
from  matplotlib import pyplot as plt
import matplotlib.image as mpimg
%matplotlib inline

In [2]:
IMG_WIDTH = 224
IMG_HEIGHT = 224
img_folder = r'C:\Users\tejas\Downloads\archive (2)\Alzheimer_s Dataset\train'

In [3]:
def create_dataset(img_folder):
   
    img_data_array=[]
    class_name=[]
   
    for dir1 in os.listdir(img_folder):
        for file in os.listdir(os.path.join(img_folder, dir1)):
       
            image_path= os.path.join(img_folder, dir1,  file)
            image= cv2.imread( image_path, cv2.COLOR_BGR2GRAY)
            image=cv2.resize(image, (IMG_HEIGHT, IMG_WIDTH),interpolation = cv2.INTER_AREA)
            image=np.array(image)
            image = image.astype('float32')
            image /= 255 
            img_data_array.append(image)
            class_name.append(dir1)
    return img_data_array, class_name
# extract the image array and class name
img_data, class_name =create_dataset(r'C:\Users\tejas\Downloads\archive (2)\Alzheimer_s Dataset\train')

In [4]:
target_dict={k: v for v, k in enumerate(np.unique(class_name))}
target_dict

{'MildDemented': 0,
 'ModerateDemented': 1,
 'NonDemented': 2,
 'VeryMildDemented': 3}

In [5]:
target_val=  [target_dict[class_name[i]] for i in range(len(class_name))]

In [6]:
X = np.array(img_data, np.float32)
X = X.flatten().reshape(-1,IMG_WIDTH*IMG_HEIGHT*1)
y = np.array(list(map(int,target_val)), np.float32)

In [7]:
print(X.shape)

(5121, 50176)


In [8]:
print(y.shape)

(5121,)


In [9]:
print(len(X))

5121


In [10]:
# def plot_image(x,y,index):
#     plt.imshow(x[index])

In [11]:
from imblearn.over_sampling import SMOTE
sm = SMOTE(sampling_strategy='minority')
X_smote, y_smote = sm.fit_resample(X, y)

In [12]:
print(len(X_smote))

7629


In [13]:
print(X_smote.shape)

(7629, 50176)


In [14]:
X_smote = np.stack((X_smote,)*3, axis=-1)

In [15]:
X_smote = X_smote.reshape(-1,224,224,3)

In [16]:
print(X_smote.shape)

(7629, 224, 224, 3)


In [25]:
vgg = VGG16(
    include_top=False,
    weights="imagenet",
    input_shape=(224,224,3))

In [26]:
for layer in vgg.layers:
  layer.trainable = False

In [27]:
x = Flatten()(vgg.output)
# x = Dense(1000, activation='relu')(x)
prediction = Dense(4, activation='softmax')(x)

In [28]:
model = Model(inputs=vgg.input, outputs=prediction)

In [29]:
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

In [30]:
model.compile(
  loss='sparse_categorical_crossentropy',
  optimizer='adam',
  metrics=['accuracy']
)

In [40]:
model.fit(X_smote,y_smote,epochs=10,validation_data=(X, y))

Epoch 1/10
239/239 [==============================] - 76s 317ms/step - loss: 0.0699 - accuracy: 0.9881 - val_loss: 0.9086 - val_accuracy: 0.7021
Epoch 2/10
239/239 [==============================] - 67s 282ms/step - loss: 0.0546 - accuracy: 0.9946 - val_loss: 0.8891 - val_accuracy: 0.7099
Epoch 3/10
239/239 [==============================] - 68s 283ms/step - loss: 0.0453 - accuracy: 0.9961 - val_loss: 0.8547 - val_accuracy: 0.7091
Epoch 4/10
239/239 [==============================] - 67s 281ms/step - loss: 0.0468 - accuracy: 0.9937 - val_loss: 0.8651 - val_accuracy: 0.7029
Epoch 5/10
239/239 [==============================] - 68s 284ms/step - loss: 0.0492 - accuracy: 0.9904 - val_loss: 1.0313 - val_accuracy: 0.7021
Epoch 6/10
239/239 [==============================] - 68s 286ms/step - loss: 0.0321 - accuracy: 0.9975 - val_loss: 0.8887 - val_accuracy: 0.7099
Epoch 7/10
239/239 [==============================] - 68s 285ms/step - loss: 0.0306 - accuracy: 0.9982 - val_loss: 0.8489 - val_ac

In [24]:
print(X_smote.shape)

(7629, 224, 224, 3)


In [26]:
print(len(y_smote))

7629


In [32]:
test_folder = r'C:\Users\tejas\Downloads\archive (2)\Alzheimer_s Dataset\test'

In [33]:
def create_dataset(img_folder):
   
    img_data_array=[]
    class_name=[]
   
    for dir1 in os.listdir(img_folder):
        for file in os.listdir(os.path.join(img_folder, dir1)):
       
            image_path= os.path.join(img_folder, dir1,  file)
            image= cv2.imread( image_path, cv2.COLOR_BGR2GRAY)
            image=cv2.resize(image, (IMG_HEIGHT, IMG_WIDTH),interpolation = cv2.INTER_AREA)
            image=np.array(image)
            image = image.astype('float32')
            image /= 255 
            img_data_array.append(image)
            class_name.append(dir1)
    return img_data_array, class_name
# extract the image array and class name
img_data, class_name =create_dataset(r'C:\Users\tejas\Downloads\archive (2)\Alzheimer_s Dataset\test')

In [34]:
target_dict={k: v for v, k in enumerate(np.unique(class_name))}
target_dict

{'MildDemented': 0,
 'ModerateDemented': 1,
 'NonDemented': 2,
 'VeryMildDemented': 3}

In [35]:
target_val=  [target_dict[class_name[i]] for i in range(len(class_name))]

In [36]:
X = np.array(img_data, np.float32)
y = np.array(list(map(int,target_val)), np.float32)

In [37]:
print(X.shape)

(1279, 224, 224)


In [38]:
X = np.stack((X,)*3, axis=-1)

In [39]:
X = X.reshape(-1,224,224,3)